In [2]:
%config IPCompleter.greedy=True

In [3]:
from pyspark import SparkContext

sc = SparkContext()

In [4]:
def parse(line):
    return [int(x) for x in line.split(",")]

In [5]:
from itertools import islice

training_data = sc.textFile('./mnist/mnist_train.csv')\
        .mapPartitionsWithIndex(
            lambda idx, it: islice(it, 1, None) if idx == 0 else it 
        )\
        .map(parse)

test_data = sc.textFile('./mnist/mnist_test.csv')\
        .mapPartitionsWithIndex(
            lambda idx, it: islice(it, 1, None) if idx == 0 else it 
        )\
        .map(parse)

In [6]:
from pyspark.mllib.clustering import KMeans, KMeansModel

In [42]:
# Build the model (cluster the data)
clusters = KMeans.train(training_data.map(lambda d: d[1:]), 10, maxIterations=10, initializationMode="random")

In [43]:
predict_cluster = test_data.map(lambda d: {"label": d[0], "cluster": clusters.predict(d[1:]), "features": d[1:]})

In [44]:
cluster_samples = predict_cluster

In [45]:
import numpy as np
import pandas as pd

In [47]:
df = pd.DataFrame(cluster_samples.collect())

cluster_count = df.groupby(['cluster', 'label']).count()

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(cluster_count)

               features
cluster label          
0       0             2
        1             1
        2            23
        3            15
        4           546
        5            59
        6             8
        7            81
        8            32
        9           520
1       0             1
        2             9
        3             3
        4             4
        5             7
        7           610
        8             9
        9            68
2       0           767
        2            14
        3             1
        5             6
        6            13
        7             1
        8             7
        9             7
3       0             2
        1          1106
        2           145
        3            58
        4            27
        5            82
        6            25
        7            76
        8            77
        9            22
4       0            28
        2            12
        3             2
        4       

In [49]:
clusters.save(sc, './mnist-kmeans')